## Part 1

In [1]:
# states are
# 0 - running
# 1 - waiting for input
# 99 - halt

mutable struct Prog
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    relative_base::Int
    state::Int
end

str2prog(s) = parse.(Int, split(s, ","))

Prog(s::String) = Prog(vcat(str2prog(s), zeros(Int, 10000)), 1, [], [], 0, 0)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        9 => 1,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(prog::Prog) = Instruction(prog.code[prog.cur])

function take(prog::Prog, mode, offset)
    if mode == 0
        return prog.code[prog.code[prog.cur + offset] + 1]
    elseif mode == 1
        return prog.code[prog.cur + offset]
    else
        return prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1]
    end
end

function update!(prog::Prog, value, offset, mode = 0)
    if mode == 0
        prog.code[prog.code[prog.cur + offset] + 1] = value
    elseif mode == 2
        prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1] = value
    end
end

function apply(prog::Prog, instruction::Instruction)
    #println(instruction)
    if instruction.op == 99
        prog.state = 99
    elseif instruction.op == 1
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 + a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 2
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 * a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 3
        if isempty(prog.input)
            prog.state = 1
        else
            update!(prog, popfirst!(prog.input), 1, instruction.modes[1])
            prog.cur += 2
        end
    elseif instruction.op == 4
        push!(prog.output, take(prog, instruction.modes[1], 1))
        prog.cur += 2
    elseif instruction.op == 5
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 != 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 6
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 == 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 7
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 < a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 8
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 == a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 9
        prog.relative_base += take(prog, instruction.modes[1], 1)
        prog.cur += 2
    end
end

function run(prog::Prog, input::Vector{Int})
    prog.input = input
    run(prog)
end

function run(prog)
    while prog.state == 0
        instruction = Instruction(prog)
        apply(prog, instruction)
    end
    
    prog.output
end

function feed(prog::Prog, input::Vector{Int})
    prog.input = vcat(prog.input, input)
    prog.state = 0
end

clear(prog) = (prog.output = [])

clear (generic function with 1 method)

In [3]:
bx = readline("input.txt") |> Prog |> run
board = Dict{Tuple{Int, Int}, Int}()
for i in 0:(div(length(bx), 3) - 1)
    board[(bx[3*i + 1], bx[3*i + 2])] = bx[3*i + 3]
end

In [9]:
println("Part 1 = ", sum(values(board) .== 2))

Part 1 = 298


In [35]:
struct Board
    tl::Tuple{Int, Int}
    br::Tuple{Int, Int}
    b::Dict{Tuple{Int, Int}, Int}
end

function Board(inp::Vector{Int})
    board = Dict{Tuple{Int, Int}, Int}()
    for i in 0:(div(length(inp), 3) - 1)
        board[(inp[3*i + 1], inp[3*i + 2])] = inp[3*i + 3]
    end
    tlx = minimum([p[1] for p in keys(board)])
    tly = minimum([p[2] for p in keys(board)])
    brx = maximum([p[1] for p in keys(board)])
    bry = maximum([p[2] for p in keys(board)])

    Board((tlx, tly), (brx, bry), board)
end

width(b::Board) = b.br[1] - b.tl[1] + 1
height(b::Board) = b.br[2] - b.tl[2] + 1

function draw(b::Board)
    chars = Dict(
        0 => ' ',
        1 => 'x',
        2 => '☐',
        3 => '_',
        4 => '⚫'
    )
    canvas = zeros(Int, height(b), width(b))
    for (k, v) in b.b
        canvas[k[2] - b.tl[2] + 1, k[1] - b.tl[1] + 1] = v
    end
    canvas = map(x -> chars[x], canvas)
    for i in 1:height(b)
        println(join(canvas[i, :]))
    end
end

draw (generic function with 1 method)

In [24]:
bx = readline("input.txt") |> Prog |> run
b = Board(bx)

Board((0, 0), (39, 25), Dict((8, 23) => 0,(21, 13) => 0,(23, 10) => 0,(14, 0) => 1,(30, 21) => 0,(4, 5) => 0,(18, 21) => 4,(6, 18) => 2,(36, 8) => 0,(1, 22) => 0…))

In [27]:
draw(b)

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
x                                      x
x         ☐☐ ☐  ☐ ☐☐☐ ☐ ☐ ☐☐   ☐☐☐ ☐   x
x ☐☐ ☐ ☐  ☐   ☐☐☐   ☐ ☐   ☐☐   ☐   ☐ ☐ x
x ☐ ☐ ☐ ☐☐☐  ☐     ☐   ☐ ☐ ☐☐ ☐☐ ☐☐☐☐  x
x      ☐☐☐☐  ☐      ☐ ☐☐  ☐☐☐☐☐☐☐ ☐☐☐☐ x
x ☐  ☐     ☐     ☐☐  ☐☐☐   ☐☐☐ ☐   ☐   x
x  ☐       ☐☐☐☐    ☐  ☐ ☐   ☐☐☐☐ ☐☐☐☐  x
x ☐☐    ☐☐ ☐   ☐☐☐ ☐   ☐     ☐☐ ☐☐☐    x
x     ☐ ☐  ☐☐☐☐ ☐☐  ☐  ☐☐ ☐☐☐  ☐☐   ☐  x
x   ☐ ☐ ☐ ☐  ☐☐☐☐   ☐ ☐ ☐☐☐  ☐☐☐   ☐   x
x  ☐   ☐☐☐ ☐   ☐☐       ☐☐ ☐ ☐ ☐☐☐☐☐   x
x  ☐☐☐☐ ☐   ☐☐☐☐ ☐ ☐☐ ☐ ☐  ☐   ☐☐   ☐  x
x ☐  ☐☐   ☐  ☐   ☐ ☐       ☐☐ ☐ ☐☐☐ ☐  x
x ☐ ☐ ☐ ☐ ☐ ☐   ☐ ☐☐ ☐☐☐☐☐  ☐☐ ☐☐☐ ☐   x
x  ☐☐   ☐    ☐  ☐☐ ☐☐☐  ☐☐☐☐☐☐ ☐ ☐☐  ☐ x
x ☐☐☐ ☐   ☐☐☐☐  ☐☐☐   ☐☐☐ ☐ ☐ ☐ ☐☐   ☐ x
x     ☐☐☐  ☐ ☐☐   ☐  ☐☐☐☐  ☐   ☐ ☐  ☐  x
x   ☐ ☐☐ ☐☐☐☐     ☐☐ ☐  ☐ ☐☐☐☐☐ ☐☐☐    x
x ☐☐☐      ☐ ☐ ☐ ☐☐     ☐ ☐   ☐     ☐  x
x                                      x
x                 ⚫                    x
x                                      x
x                                      x
x               

In [32]:
code = collect(readline("input.txt"))
code[1] = '2'
prog = Prog(join(code))
bx = run(prog)
b = Board(bx)

Board((-1, 0), (39, 25), Dict((8, 23) => 0,(21, 13) => 0,(23, 10) => 0,(14, 0) => 1,(30, 21) => 0,(4, 5) => 0,(18, 21) => 4,(6, 18) => 2,(36, 8) => 0,(1, 22) => 0…))

In [36]:
draw(b)

 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 x                                      x
 x         ☐☐ ☐  ☐ ☐☐☐ ☐ ☐ ☐☐   ☐☐☐ ☐   x
 x ☐☐ ☐ ☐  ☐   ☐☐☐   ☐ ☐   ☐☐   ☐   ☐ ☐ x
 x ☐ ☐ ☐ ☐☐☐  ☐     ☐   ☐ ☐ ☐☐ ☐☐ ☐☐☐☐  x
 x      ☐☐☐☐  ☐      ☐ ☐☐  ☐☐☐☐☐☐☐ ☐☐☐☐ x
 x ☐  ☐     ☐     ☐☐  ☐☐☐   ☐☐☐ ☐   ☐   x
 x  ☐       ☐☐☐☐    ☐  ☐ ☐   ☐☐☐☐ ☐☐☐☐  x
 x ☐☐    ☐☐ ☐   ☐☐☐ ☐   ☐     ☐☐ ☐☐☐    x
 x     ☐ ☐  ☐☐☐☐ ☐☐  ☐  ☐☐ ☐☐☐  ☐☐   ☐  x
 x   ☐ ☐ ☐ ☐  ☐☐☐☐   ☐ ☐ ☐☐☐  ☐☐☐   ☐   x
 x  ☐   ☐☐☐ ☐   ☐☐       ☐☐ ☐ ☐ ☐☐☐☐☐   x
 x  ☐☐☐☐ ☐   ☐☐☐☐ ☐ ☐☐ ☐ ☐  ☐   ☐☐   ☐  x
 x ☐  ☐☐   ☐  ☐   ☐ ☐       ☐☐ ☐ ☐☐☐ ☐  x
 x ☐ ☐ ☐ ☐ ☐ ☐   ☐ ☐☐ ☐☐☐☐☐  ☐☐ ☐☐☐ ☐   x
 x  ☐☐   ☐    ☐  ☐☐ ☐☐☐  ☐☐☐☐☐☐ ☐ ☐☐  ☐ x
 x ☐☐☐ ☐   ☐☐☐☐  ☐☐☐   ☐☐☐ ☐ ☐ ☐ ☐☐   ☐ x
 x     ☐☐☐  ☐ ☐☐   ☐  ☐☐☐☐  ☐   ☐ ☐  ☐  x
 x   ☐ ☐☐ ☐☐☐☐     ☐☐ ☐  ☐ ☐☐☐☐☐ ☐☐☐    x
 x ☐☐☐      ☐ ☐ ☐ ☐☐     ☐ ☐   ☐     ☐  x
 x                                      x
 x                 ⚫                    x
 x                                      x
 x                                

In [39]:
feed(prog, [-1])
bx = run(prog)
b = Board(bx)
draw(b)

 xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
 x                                      x
 x         ☐☐ ☐  ☐ ☐☐☐ ☐ ☐ ☐☐   ☐☐☐ ☐   x
 x ☐☐ ☐ ☐  ☐   ☐☐☐   ☐ ☐   ☐☐   ☐   ☐ ☐ x
 x ☐ ☐ ☐ ☐☐☐  ☐     ☐   ☐ ☐ ☐☐ ☐☐ ☐☐☐☐  x
 x      ☐☐☐☐  ☐      ☐ ☐☐  ☐☐☐☐☐☐☐ ☐☐☐☐ x
 x ☐  ☐     ☐     ☐☐  ☐☐☐   ☐☐☐ ☐   ☐   x
 x  ☐       ☐☐☐☐    ☐  ☐ ☐   ☐☐☐☐ ☐☐☐☐  x
 x ☐☐    ☐☐ ☐   ☐☐☐ ☐   ☐     ☐☐ ☐☐☐    x
 x     ☐ ☐  ☐☐☐☐ ☐☐  ☐  ☐☐ ☐☐☐  ☐☐   ☐  x
 x   ☐ ☐ ☐ ☐  ☐☐☐☐   ☐ ☐ ☐☐☐  ☐☐☐   ☐   x
 x  ☐   ☐☐☐ ☐   ☐☐       ☐☐ ☐ ☐ ☐☐☐☐☐   x
 x  ☐☐☐☐ ☐   ☐☐☐☐ ☐ ☐☐ ☐ ☐  ☐   ☐☐   ☐  x
 x ☐  ☐☐   ☐  ☐   ☐ ☐       ☐☐ ☐ ☐☐☐ ☐  x
 x ☐ ☐ ☐ ☐ ☐ ☐   ☐ ☐☐ ☐☐☐☐☐  ☐☐ ☐☐☐ ☐   x
 x  ☐☐   ☐    ☐  ☐☐ ☐☐☐  ☐☐☐☐☐☐ ☐ ☐☐  ☐ x
 x ☐☐☐ ☐   ☐☐☐☐  ☐☐☐   ☐☐☐ ☐ ☐ ☐ ☐☐   ☐ x
 x     ☐☐☐  ☐ ☐☐   ☐  ☐☐☐☐  ☐   ☐ ☐  ☐  x
 x   ☐ ☐☐ ☐☐☐☐     ☐☐ ☐  ☐ ☐☐☐☐☐ ☐☐☐    x
 x ☐☐☐      ☐ ☐ ☐ ☐☐     ☐ ☐   ☐     ☐  x
 x                                      x
 x                                      x
 x                  ⚫                   x
 x                                

In [18]:
for (k, v) in d
    println(k, ":", v)
end

3:4
1:2
